# Deploy <font color='#6957FF'>COMET Quality Estimation</font> Model Package from AWS Marketplace


**COMET (Cross-lingual Optimized Metric for Evaluation of Translation)** is a neural framework for training multilingual machine translation evaluation models. This package provides a commercially-friendly offering of the **wmt22-cometkiwi-da** model, a direct assessment (DA) model trained on WMT22 data and [made available at HuggingFace](https://huggingface.co/Unbabel/wmt22-cometkiwi-da) for non-commercial use.

This sample notebook shows you how to deploy <font color='#C9FF33'>[COMETKiwi](https://aws.amazon.com/marketplace/pp/prodview-k5lgwkzc62j5u)</font> using Amazon SageMaker.

> **Note**: This model package only supports SageMaker Realtime Inference.

## Pre-requisites:
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
2. To deploy this ML model successfully, ensure that:
    - Either your IAM role has these three permissions, and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        - **aws-marketplace:ViewSubscriptions**
        - **aws-marketplace:Unsubscribe**
        - **aws-marketplace:Subscribe**  
    - or your AWS account has a subscription to <font color='#C9FF33'>[COMETKiwi](https://aws.amazon.com/marketplace/pp/prodview-k5lgwkzc62j5u)</font>. If so, skip step [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
3. Before running this notebook, please make sure you are using a virtualenv with the notebook so that the dependencies are installed in the virtual env.
4. Create a `.env` file from `.env_example` and set the appropriate values for AWS credentials and for AWS Region.

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
3. [Example use cases](#3.-Example-use-cases)
4. [Clean-up](#4.-Clean-up)

## Usage instructions
You can run this notebook one cell at a time (by using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='#C9FF33'>[COMETKiwi](https://aws.amazon.com/marketplace/pp/prodview-k5lgwkzc62j5u)</font> in AWS Marketplace.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, [see their documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
!pip install sagemaker~=2.235.0 python-dotenv~=1.0.0
import json
from sagemaker import ModelPackage, Predictor, Session
from sagemaker import deserializers, get_execution_role, serializers

In [ ]:
import dotenv
dotenv.load_dotenv()

In [2]:
# Set model package ARN
region = "us-east-1"
model_package_arn = f"arn:aws:sagemaker:{region}:571600842260:model-package/comet-wmt22-cometkiwi-da"

Make sure that your default execution role has permissions necessary as specified in the pre-requisites or set the value of role directly to the suitable role.

In [ ]:
session = Session()
role = get_execution_role(session)  # Or directly specify a suitable role

In [7]:
# Choose an instance type to run the model
instance_type = "ml.m5.xlarge"  # Adjust based on your needs

In [8]:
model_name = model_package_arn.split("/")[-1]

# Create a deployable model from the model package
model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=session
)

print(f"{model_name=}")

model_name='comet-wmt22-cometkiwi-da'


### A. Create an endpoint
This process takes several minutes.

In [ ]:
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=model_name,
    container_startup_health_check_timeout=300,
)

### B. Create input payload

In [19]:
# Example input for quality estimation
sample_input = {
    "data": [
        {
            "src": "The cat sat on the mat.",
            "mt": "Le chat était assis sur le tapis.",
        },
        {
            "src": "I love programming in Python.",
            "mt": "J'aime programmer en Python.",
        },
        {
            "src": "The weather is beautiful today.",
            "mt": "Il fait beau aujourd'hui.",
        }
    ],
    "batch_size": 3
}

### C. Perform real-time inference

In [20]:
predictor = Predictor(
    endpoint_name=model_name,
    sagemaker_session=session,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

response = predictor.predict(sample_input)
print(json.dumps(response, indent=2))

{
  "scores": [
    0.8695598840713501,
    0.8979955315589905,
    0.8807777762413025
  ],
  "system_score": 0.882777730623881,
  "message": "Prediction successful"
}


## 3. Example use cases

### Multiple language pairs
COMET can evaluate translations across multiple language pairs in a single batch:

In [21]:
multilingual_input = {
    "data": [
        {
            "src": "A group of researchers has launched a new model for translation-related tasks.",
            "mt": "Un grupo de investigadores ha lanzado un nuevo modelo para tareas relacionadas con la traducción.",
        },
        {
            "src": "The new model is based on a transformer architecture.",
            "mt": "Le nouveau modèle est basé sur une architecture de transformateur.",
        }
    ],
    "batch_size": 2
}

response = predictor.predict(multilingual_input)
print(json.dumps(response, indent=2))

{
  "scores": [
    0.8924700021743774,
    0.8798026442527771
  ],
  "system_score": 0.8861363232135773,
  "message": "Prediction successful"
}


### Batch Processing
You can process multiple translations at once by adjusting the batch size:

In [23]:
batch_input = {
    "data": [
        {
            "src": text,
            "mt": translation,
        } for text, translation in [
            (
                "The cat sat on the mat.",
                "Le chat était assis sur le tapis.",
            ),
            (
                "I love programming in Python.",
                "J'aime programmer en Python.",
            ),
            (
                "The weather is beautiful today.",
                "Il fait beau aujourd'hui.",
            )
        ]
    ],
    "batch_size": 3
}

response = predictor.predict(batch_input)
print(json.dumps(response, indent=2))

{
  "scores": [
    0.8695598840713501,
    0.8979955315589905,
    0.8807777762413025
  ],
  "system_score": 0.882777730623881,
  "message": "Prediction successful"
}


## 4. Clean-up

Once you are done using the endpoint, to avoid unnecessary costs, you can delete it and the model by running the following commands:

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.